In [1]:
"""
Relevance Detection
"""

'\nRelevance Detection\n'

In [2]:
import pandas as pd
import nltk
import numpy as np
import preprocessing
import importlib
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import Counter

In [63]:
#run this cell to reload the preprocessing module
importlib.reload(preprocessing)

<module 'preprocessing' from '/Users/dannyyang/Documents/GitHub/Insights-FakeNews/preprocessing.py'>

In [4]:
train_stances = pd.read_csv("fn_data/train_stances.csv")
train_stances["Stance"] = train_stances["Stance"].apply(lambda x: "related" if x != "unrelated" else x)
print(train_stances.shape)
train_stances.head()

(49972, 3)


,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,related
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,related


In [5]:
train_bodies = pd.read_csv("fn_data/train_bodies.csv")
print(train_bodies.shape)
train_bodies.head()

(1683, 2)


,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [29]:
idx = np.random.permutation(np.arange(len(train_stances))) 
stances = train_stances.values[idx]
train = int(len(stances)*0.8)
stances_tr = stances[:train]
stances_val = stances[train:]

In [37]:
def get_body_list(data):
    return [preprocessing.get_body(i, train_bodies) for i in set([x[1] for x in data])]

In [43]:
body_list = get_body_list(stances_tr)

In [44]:
corpus = [preprocessing.get_clean_tokens(x) for x in body_list]

In [65]:
idf = preprocessing.build_idf_tokens(corpus)

In [66]:
body = preprocessing.get_body(6, train_bodies)
processed = preprocessing.process_body(body, idf)
print(body, processed['common_nouns'],processed['common_verbs'],processed['common_bigrams'])

NameError: name 'n_tfidf' is not defined